In [10]:
import mysql.connector
import pandas as pd

# Koneksi ke database
def establish_connection():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='data_smartphone',
        port=3306
    )

# Eksekusi query dan konversi ke DataFrame
def execute_query_to_dataframe(connection, query):
    try:
        cursor = connection.cursor(dictionary=True)
        cursor.execute(query)
        result = cursor.fetchall()
        return pd.DataFrame(result)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        cursor.close()

# Contoh penggunaan
connection = establish_connection()
if connection.is_connected():
    print("Koneksi ke database berhasil")
    query = """
    SELECT
        data.toko as toko,
        data.Merek,
        data.Tipe,
        data.Bulan,
        data.Jumlah_Stok,
        data.Jumlah_Terjual,
        data.Harga_Satuan_Rp,
        data.Total_Penjualan_Rp,
        data_spesifikasi.Kamera_Utama_MP,
        data_spesifikasi.Kamera_Depan_MP,
        data_spesifikasi.RAM,
        data_spesifikasi.Memori_Internal,
        data_spesifikasi.Baterai_mAh,
        data_spesifikasi.Jenis_Layar
    FROM 
        data
    LEFT JOIN 
        data_spesifikasi 
    ON 
        data.Tipe = data_spesifikasi.Tipe
    WHERE 
        data.toko = 'TM Store'
    """
    df = execute_query_to_dataframe(connection, query)
    display(df.head())
else:
    print("No connection to the database.")


Koneksi ke database berhasil


,toko,Merek,Tipe,Bulan,Jumlah_Stok,Jumlah_Terjual,Harga_Satuan_Rp,Total_Penjualan_Rp,Kamera_Utama_MP,Kamera_Depan_MP,RAM,Memori_Internal,Baterai_mAh,Jenis_Layar
0,TM Store,Realme,8 Pro,Jan,18,9,4999000,44991000,64.0,32.0,8GB,128GB,4500.0,AMOLED
1,TM Store,Realme,8 Pro,Jan,18,9,4999000,44991000,108.0,16.0,8GB,128GB,4500.0,Super AMOLED
2,TM Store,Realme,8 Pro,Feb,15,4,4999000,19996000,64.0,32.0,8GB,128GB,4500.0,AMOLED
3,TM Store,Realme,8 Pro,Feb,15,4,4999000,19996000,108.0,16.0,8GB,128GB,4500.0,Super AMOLED
4,TM Store,Realme,8 Pro,Mar,21,1,4999000,4999000,64.0,32.0,8GB,128GB,4500.0,AMOLED


In [11]:
import numpy as np

# Menangani nilai yang hilang
def handle_missing_values(df):
    df.fillna({'': 0, np.number: 0}, inplace=True)

# Kategorisasi persentase penjualan
def categorize_sales_percentage(percent_sold):
    if percent_sold < 10:
        return 'Sangat Rendah'
    elif percent_sold < 20:
        return 'Rendah'
    elif percent_sold < 50:
        return 'Cukup'
    elif percent_sold < 70:
        return 'Tinggi'
    else:
        return 'Sangat Tinggi'

# Persiapan DataFrame
if df is not None and not df.empty:
    columns_needed = ['Merek', 'Tipe', 'Jumlah_Stok', 'Jumlah_Terjual',
                      'Harga_Satuan_Rp', 'Total_Penjualan_Rp',
                      'Kamera_Utama_MP', 'Kamera_Depan_MP', 'RAM',
                      'Memori_Internal', 'Baterai_mAh', 'Jenis_Layar']

    df_klastering = df[columns_needed].copy()
    df_klastering['Persentase_Jumlah_Terjual'] = (df_klastering['Jumlah_Terjual'] / df_klastering['Jumlah_Stok']) * 100

    handle_missing_values(df_klastering)
    df_klastering['Kategori_Penjualan'] = df_klastering['Persentase_Jumlah_Terjual'].apply(categorize_sales_percentage)

    display(df_klastering.head())
else:
    print("No data available for processing.")


,Merek,Tipe,Jumlah_Stok,Jumlah_Terjual,Harga_Satuan_Rp,Total_Penjualan_Rp,Kamera_Utama_MP,Kamera_Depan_MP,RAM,Memori_Internal,Baterai_mAh,Jenis_Layar,Persentase_Jumlah_Terjual,Kategori_Penjualan
0,Realme,8 Pro,18,9,4999000,44991000,64.0,32.0,8GB,128GB,4500.0,AMOLED,50.000000,Tinggi
1,Realme,8 Pro,18,9,4999000,44991000,108.0,16.0,8GB,128GB,4500.0,Super AMOLED,50.000000,Tinggi
2,Realme,8 Pro,15,4,4999000,19996000,64.0,32.0,8GB,128GB,4500.0,AMOLED,26.666667,Cukup
3,Realme,8 Pro,15,4,4999000,19996000,108.0,16.0,8GB,128GB,4500.0,Super AMOLED,26.666667,Cukup
4,Realme,8 Pro,21,1,4999000,4999000,64.0,32.0,8GB,128GB,4500.0,AMOLED,4.761905,Sangat Rendah


In [12]:
from sklearn.cluster import Birch

# Inisialisasi dan penerapan model BIRCH
birch_model = Birch(n_clusters=5, threshold=0.5)
df_features = df_klastering[['Jumlah_Stok', 'Jumlah_Terjual', 'Persentase_Jumlah_Terjual']]
birch_model.fit(df_features)

clusters = birch_model.predict(df_features)
df_klastering['Cluster'] = clusters
centroids = birch_model.subcluster_centers_

display(df_klastering.head())


,Merek,Tipe,Jumlah_Stok,Jumlah_Terjual,Harga_Satuan_Rp,Total_Penjualan_Rp,Kamera_Utama_MP,Kamera_Depan_MP,RAM,Memori_Internal,Baterai_mAh,Jenis_Layar,Persentase_Jumlah_Terjual,Kategori_Penjualan,Cluster
0,Realme,8 Pro,18,9,4999000,44991000,64.0,32.0,8GB,128GB,4500.0,AMOLED,50.000000,Tinggi,1
1,Realme,8 Pro,18,9,4999000,44991000,108.0,16.0,8GB,128GB,4500.0,Super AMOLED,50.000000,Tinggi,1
2,Realme,8 Pro,15,4,4999000,19996000,64.0,32.0,8GB,128GB,4500.0,AMOLED,26.666667,Cukup,3
3,Realme,8 Pro,15,4,4999000,19996000,108.0,16.0,8GB,128GB,4500.0,Super AMOLED,26.666667,Cukup,3
4,Realme,8 Pro,21,1,4999000,4999000,64.0,32.0,8GB,128GB,4500.0,AMOLED,4.761905,Sangat Rendah,2


In [13]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def visualize_clusters(df, centroids, clusters):
    # Mapping warna untuk kategori penjualan
    color_mapping = {
        'Sangat Rendah': 'blue',
        'Rendah': 'green',
        'Cukup': 'yellow',
        'Tinggi': 'orange',
        'Sangat Tinggi': 'black'
    }

    # Buat DataFrame visualisasi
    df['Cluster'] = clusters
    df['Color'] = df['Kategori_Penjualan'].map(color_mapping)

    # Hitung rata-rata fitur untuk setiap klaster
    cluster_means = df.groupby('Cluster')[['Jumlah_Stok', 'Jumlah_Terjual', 'Persentase_Jumlah_Terjual']].mean()

    # Plot dengan subplots
    fig = make_subplots(rows=2, cols=2, 
                        specs=[[{'type': 'xy'}, {'type': 'xy'}],
                               [{'type': 'xy'}, {'type': 'domain'}]],
                        subplot_titles=('Jumlah Stok vs Jumlah Terjual',
                                        'Jumlah Stok vs Persentase Terjual',
                                        'Jumlah Terjual vs Persentase Terjual',
                                        'Distribusi Klaster'),
                        vertical_spacing=0.1, horizontal_spacing=0.1)

    # Scatter plots untuk data points
    scatter_plots = [
        ('Jumlah_Stok', 'Jumlah_Terjual', 1, 1),
        ('Jumlah_Stok', 'Persentase_Jumlah_Terjual', 1, 2),
        ('Jumlah_Terjual', 'Persentase_Jumlah_Terjual', 2, 1)
    ]

    for x, y, row, col in scatter_plots:
        fig.add_trace(
            go.Scatter(
                x=df[x], y=df[y],
                mode='markers',
                marker=dict(size=8, color=df['Color'], opacity=0.6),
                text=df['Merek'] + ' - ' + df['Tipe'],
                hoverinfo='text',
                showlegend=False
            ),
            row=row, col=col
        )

        # Add centroids
        fig.add_trace(
            go.Scatter(
                x=cluster_means[x],
                y=cluster_means[y],
                mode='markers',
                marker=dict(size=15, color='black', symbol='star'),
                name='Centroids',
                showlegend=False
            ),
            row=row, col=col
        )

    # Pie chart for cluster distribution
    cluster_counts = df['Cluster'].value_counts().sort_index()
    fig.add_trace(
        go.Pie(
            labels=[f'Cluster {i}' for i in cluster_counts.index],
            values=cluster_counts.values,
            name='Cluster Distribution'
        ),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        title_text="Visualisasi Klaster BIRCH",
        height=1000,
        width=1200,
        showlegend=True
    )

    # Add color legend
    for category, color in color_mapping.items():
        fig.add_trace(go.Scatter(
            x=[None], y=[None], 
            mode='markers', 
            marker=dict(size=10, color=color),
            legendgroup="Kategori Penjualan", 
            showlegend=True, 
            name=category
        ))

    fig.show()

visualize_clusters(df_klastering, centroids, clusters)


In [14]:
from sklearn.manifold import TSNE

def visualize_clusters_tsne(df, clusters):
    features = ['Jumlah_Stok', 'Jumlah_Terjual', 'Persentase_Jumlah_Terjual']
    df_numeric = df[features].copy()
    
    # Konversi kolom ke tipe numerik dan tangani nilai yang hilang
    for col in df_numeric.columns:
        df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
    
    df_numeric = df_numeric.fillna(df_numeric.median())
    X_normalized = (df_numeric.values - df_numeric.values.mean(axis=0)) / df_numeric.values.std(axis=0)
    
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X_normalized)
    
    df_plot = df[['Merek', 'Tipe', 'Kategori_Penjualan']].copy()
    df_plot['x'] = X_tsne[:, 0]
    df_plot['y'] = X_tsne[:, 1]
    df_plot['Cluster'] = clusters
    
    color_map = {
        0: '#FF4136',  
        1: '#2ECC40',  
        2: '#0074D9',  
        3: '#FF851B',  
        4: '#B10DC9',  
    }
    
    fig = go.Figure()

    for cluster in range(5):  
        cluster_data = df_plot[df_plot['Cluster'] == cluster]
        fig.add_trace(go.Scatter(
            x=cluster_data['x'],
            y=cluster_data['y'],
            mode='markers',
            marker=dict(size=8, color=color_map[cluster], opacity=0.7),
            text=cluster_data['Merek'] + ' - ' + cluster_data['Tipe'] + '<br>Kategori: ' + cluster_data['Kategori_Penjualan'],
            hoverinfo='text',
            name=f'Cluster {cluster}'
        ))

    fig.update_layout(
        title='Visualisasi 5 Cluster Utama BIRCH menggunakan t-SNE',
        xaxis_title='t-SNE Dimension 1',
        yaxis_title='t-SNE Dimension 2',
        legend_title='Clusters',
        height=800,
        width=1000,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(size=12)
    )

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

    fig.show()

    cluster_stats = df_plot.groupby('Cluster').agg({
        'Merek': 'count',
        'Kategori_Penjualan': lambda x: x.value_counts().index[0]
    }).rename(columns={'Merek': 'Jumlah Data', 'Kategori_Penjualan': 'Kategori Dominan'})
    
    display(cluster_stats)

visualize_clusters_tsne(df_klastering, clusters)


,Jumlah Data,Kategori Dominan
Cluster,,
0,475,Sangat Tinggi
1,523,Tinggi
2,463,Rendah
3,467,Cukup
4,520,Cukup


In [15]:
def visualize_clusters_3d(df, clusters):
    features = ['Jumlah_Stok', 'Jumlah_Terjual', 'Persentase_Jumlah_Terjual']
    df_numeric = df[features].copy()
    
    for col in df_numeric.columns:
        df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
    
    df_numeric = df_numeric.fillna(df_numeric.median())
    X_normalized = (df_numeric.values - df_numeric.values.mean(axis=0)) / df_numeric.values.std(axis=0)
    
    tsne = TSNE(n_components=3, random_state=42)
    X_tsne = tsne.fit_transform(X_normalized)
    
    df_plot = df[['Merek', 'Tipe', 'Kategori_Penjualan']].copy()
    df_plot['x'] = X_tsne[:, 0]
    df_plot['y'] = X_tsne[:, 1]
    df_plot['z'] = X_tsne[:, 2]
    df_plot['Cluster'] = clusters
    
    color_map = {
        0: '#FF4136',  
        1: '#2ECC40',  
        2: '#0074D9',  
        3: '#FF851B',  
        4: '#B10DC9',  
    }
    
    fig = go.Figure()

    for cluster in range(5):  
        cluster_data = df_plot[df_plot['Cluster'] == cluster]
        fig.add_trace(go.Scatter3d(
            x=cluster_data['x'],
            y=cluster_data['y'],
            z=cluster_data['z'],
            mode='markers',
            marker=dict(size=4, color=color_map[cluster], opacity=0.7),
            text=cluster_data['Merek'] + ' - ' + cluster_data['Tipe'] + '<br>Kategori: ' + cluster_data['Kategori_Penjualan'],
            hoverinfo='text',
            name=f'Cluster {cluster}'
        ))

    fig.update_layout(
        title='Visualisasi 3D 5 Cluster Utama BIRCH menggunakan t-SNE',
        scene=dict(
            xaxis_title='t-SNE Dimension 1',
            yaxis_title='t-SNE Dimension 2',
            zaxis_title='t-SNE Dimension 3',
        ),
        legend_title='Clusters',
        height=800,
        width=1000,
        margin=dict(r=20, b=10, l=10, t=40)
    )

    fig.show()

    cluster_stats = df_plot.groupby('Cluster').agg({
        'Merek': 'count',
        'Kategori_Penjualan': lambda x: x.value_counts().index[0]
    }).rename(columns={'Merek': 'Jumlah Data', 'Kategori_Penjualan': 'Kategori Dominan'})
    
    display(cluster_stats)

visualize_clusters_3d(df_klastering, clusters)


,Jumlah Data,Kategori Dominan
Cluster,,
0,475,Sangat Tinggi
1,523,Tinggi
2,463,Rendah
3,467,Cukup
4,520,Cukup
